# 8章 ディープラーニング
## 8.1 ネットワークをより深く

### 8.1.1 よりディープなネットワークへ
* VGGと呼ばれるネットワークを参考に，よりディープなネットワークを作る
  * 層がより深くなった
  * フィルターは3$\times$3の小さなフィルターを使う
  * 層が深くなるにつれ，チャンネル数を大きくする
    * 畳み込み層のチャンネル数が16,16,32,32,64,64と増えていく
  * 途中でプーリング層を挿入し，中間データの空間サイズを徐々に小さくする
  * 全結合層ではDropoutレイヤを使用する
  * 活性化関数は ReLU を使用する
  * 重みの初期値としてHeの初期値を使用する
  * 重みパラメータの更新はAdamの手法を使う

In [5]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
sys.path.append(os.pardir + "/deep_learning_from_scratch")

import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *

import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

In [6]:
# ディープなネットワークの実装コード(ch08/deep_convnet.pyより)

class DeepConvNet:
    """認識率99%以上の高精度なConvNet

    ネットワーク構成は下記の通り
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 重みの初期化===========
        # 各層のニューロンひとつあたりが、前層のニューロンといくつのつながりがあるか（TODO:自動で計算する）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLUを使う場合に推奨される初期値
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # レイヤの生成===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

### 8.1.2 さらに認識精度を高めるには
* Webサイト「[What is the class of this image?](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html)」には様々なデータセットを対象に論文で発表されてきた数々の手法の精度がランキングで掲載されている
  * ランキングの上位はCNNをベースとしたものが多い
  * 文字認識などの比較的単純な問題の場合，それほど層が深くないCNNでも成績が高い．層を深くすることの精度向上へ貢献度合いは，複雑な問題の方が高い
  * 精度を高めるための技術についてヒントを得られる．たとえばアンサンブル学習とか，学習係数の減衰とか，データ拡張(Data Augmentation)とか

#### データ拡張（Data Augmentation）
* 入力画像(訓練用画像)をアルゴリズムで人工的に拡張する
  * 入力画像に対して回転や移動などの微小変化を与えて画像枚数を増やす．データセットが限られる場合には有効な手法
* 画像の拡張方法も色々ある
  * crop処理（一部を切り出す），flip処理（左右反転させる），輝度を変化させる，拡大・縮小する，など

### 8.1.3 層を深くすることのモチベーション
* 深くすることの重要性は，理論的にあまりわかっていない
* ILSVRCなどのコンペの結果から重要性を汲み取ることができる
  * 上位を占める手法の多くがディープラーニングで，層をより深くする方向に向かっている
* 層を深くする利点
  * 層を深くしなかった場合に比べてより少ないパラメータで同等以上の表現力を達成できる
    * 出力データの特定要素の値に影響を与える入力データの要素の範囲は，5x5のフィルターの畳み込み1層と3x3の畳み込み2層とで同等．後者の方がパラメータ数が少ない（前者は25個，後者は18個）
  * 受容野(receptive field, ニューロンに変化を生じさせる局所的な空間領域)を広くカバーできる
  * 層を深くするとReLUのような非線形の活性化関数が畳み込みの間に挟まれることにより表現力が向上する
  * 層を深くすることで学習データを少なくできる高速に学習できる
* ただし，層を深くできるのは，正しく学習できる技術や計算機能力の向上が前提として存在するからということに注意

## 8.2 ディープラーニングの小歴史
* 最近のディープラーニングのトレンドについて
* ディープラーニングが注目を集めるきっかけ: [ILSVRC(ImageNet Large Scale Visual Recognition Challenge)](http://www.image-net.org/challenges/LSVRC/)のAlexNetが2012年にディープラーニング手法で圧倒的成績で優勝したこと

### 8.2.1 [ImageNet](http://www.image-net.org)
* 画像データセット．
* 様々な種類の，100万枚を超える画像を含む．
* 画像にはラベル(クラス名)が紐づけられている
* ILSVRCというコンペではこのデータセットを使用している
  * 様々なテスト項目があり，その一つに「クラス分類(classification)」がある
  * 最近の結果
    * 2012年を境にディープラーニングが常にトップ
    * 2012年以降，着実に精度が改善されている
    * 2015年のResNetは誤認識率3.5%を達成し，人間の認識能力を上回ったとまで言われた

### 8.2.2 VGG
* CNNの一つ．
* 畳み込み層とプーリング層から構成される
* 階層を16層(VGG16)または19層(VGG19)まで深くしている
* 3$\times$3の小さなフィルタによる畳み込みが連続している点が特徴的
  * 畳み込み層が2回から4回連続し，プーリング層でサイズを半分にするという処理を繰り返し行う
* シンプルな構成であり，応用性が高いためVGGベースのネットワークは好まれる傾向にある

### 8.2.3 GoogLeNet
* 基本構成はCNNだが，ネットワークが縦方向だけでなく横方向にも広がりを持つ点が特徴的
  * インセプション構造と呼ばれる
* インセプション構造とは
  * サイズの異なるフィルタとプーリングを複数適用し，その結果を結合する
  * この構造を一つの構成要素として使用するのが，GoogLeNetの特徴
* サイズが1$\times$1のフィルタの畳み込み層を多くの場所で使用する
  * この畳み込み演算はチャネル方向にサイズを減らすことでパラメータの削減や処理を高速化している

### 8.2.4 ResNet
* Microsoftのチームによって開発されたネットワーク
* VGGをベースとしてスキップ構造を取り入れ，層を深くしたもの
* スキップ構造：層を深くしても学習できる仕掛け
  * 畳み込みをスキップした入力データを出力に合算する
  * 逆伝播時も上流からの勾配をそのまま下流に流すことになる
    * これにより，上流からの勾配に手を加えずそのまま流すことになるため，勾配が消失するのを軽減することが期待できる
* ILSVRCのコンペでは誤認識率3.5%という驚異的な結果を得た

#### 転移学習
* 学習済みの重みを別のニューラルネットワークにコピーして再学習(fine tuning)を行うこと
* 手元のデータセットが少ない場合は有効な手法

## 8.3 ディープラーニングの高速化
### 8.3.1 取り組むべき問題
* AlexNetのforward処理の例
  * 多くの時間が畳み込み層の計算に費やされている
    * 推論時だけでなく学習時も同様
  * この演算をいかに高速に行うかが課題
  * 畳み込み層の演算は基本的に積和演算なので，これを如何に効率よく計算するかという話

### 8.3.2 GPUによる高速化
* GPUコンピューティング
  * 並列的な数値演算を高速に行うことができる能力を様々な用途に活用する
  * ディープラーニングでは大量の積和演算が必要になり，その演算にGPUを活用可能
  * 前章で扱うim2colのように演算を大きな行列の内積に変換することはGPUにとって都合のよい計算方法
* AlexNetの事例では，GPU活用により40日かかる学習をGPUで6日に削減した
  * cuDNNというディープラーニングに最適化したライブラリの併用でさらに高速化
* GPUはNVIDIAとAMDが主に提供しているが，NVIDIAの方がディープラーニングで使用するには都合が良い
  * ディープラーニングで使われるGPUコンピューティング向け開発環境CUDAがNVIDIA提供

### 8.3.3 分散学習
* ディープラーニングの学習をスケールアウトさせようという考え
* 必要性
  * ディープラーニングの演算は時間がかかる（構造が複雑だと数週間に及ぶことも）
  * ディープラーニングは多くの試行錯誤が必要
  * したがって，1回の学習にかける時間を小さくしたい
* 方法
  * 一つの計算を複数台で分散して行う
  * TensorFlow(Google)やCNTK(Microsoft)は分散学習を重要視している
* 効果（TensorFlowによる事例）
  * GPU数が増えるにつれ学習速度も向上する
  * 100個のGPUで56倍の高速化（例えば7日の処理が3時間で終了するということ）
* 課題
  * どのように計算を分散させるかはむずかしい問題
  * マシン間の通信やデータ同期など

### 8.3.4 演算精度のビット削減
* 高速化においては計算量の面以外にメモリ容量やバス帯域などがボトルネックになり得る
  * 大量のパラメータや中間データを保持するためにメモリが必要
  * コアおよびGPU間の通信，キャッシュの利用などによりバスを流れるデータ量が増加する
* データのビット数はできるだけ小さくしたい
  * 計算精度のために32ビットや64ビットの浮動小数点数を使うと計算コストやメモリ使用量，バス帯域への負荷が大きい
  * ディープラーニングでは，数値精度のビット数をそれほど必要しないことがわかっている
    * ニューラルネットワークのロバスト性による
  * コンピュータ上での処理においては16ビットの半精度浮動小数点数で十分であることが分かっている
* 実行環境における16ビット演算サポート
  * NVIDIAの次世代GPU（Pascalアーキテクチャ）では半精度浮動小数点演算がサポートされる
    * 従来の実行環境では演算自体は16ビットで行われてこなかった
  * Pythonでは通常64ビットの浮動小数点演算が行われる
  * NumPyでは16ビット半精度浮動小数点数のデータ型が存在する
    * ただし内部の演算は16ビットで行われていない
* 動向
  * Binarized Neural Networks：重みや中間データを1ビットで表現する
  * 組み込み向けなどで利用する場合を考えると，今後目が離せない分野

## 8.4 ディープラーニングの実用例
* 物体認識以外にも様々な問題に適用することができる

### 8.4.1 物体検出
* 画像中から物体の位置の特定を含めてクラス分類を行う問題
* 位置を特定することや，複数物体が存在する可能性を含めて処理する必要がある
* 有名な手法：R-CNN
  1. 画像の入力
  2. 候補領域抽出(Extract region proposals)
    * オブジェクトらしい領域を探し出す
    * 手法として例えばSelective Searchなど
    * CNNを使って抽出を行う Faster R-CNNという手法もある
  3. CNN特徴の計算(Compute CNN features)
    * 探し出した領域に対してCNNを適用する
    * 領域を正方形に変形
  4. 領域の分類(Classify regions)
    * その領域のクラス分類を行う
    * SVMを使用する

### 8.4.2 セグメンテーション
* 画像に対してピクセルレベルでクラス分類を行う問題
* 教師データにはピクセル単位でオブジェクトごとに色付けされたデータを使う
* 推論の際には入力画像のすべてのピクセルに対してクラス分類を行う
* ピクセルを分類するには
  * もっとも単純な方法：すべてのピクセルごとに推論処理を行う
    * ピクセルの処理だけフォワーディング処理が必要，畳み込みでは多くの再計算が必要
  * [FCN(Fully Convolutional Network)](https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf)
    * 一般的なCNNが全結合層を含むのに対しFCNでは全結合層を「同じ働きをする畳み込み層に置き換える」
    * 小さくなった中間データを，最後にバイリニア補間による拡大により入力データと同じサイズまで一気に拡大する
      * FCNではこの拡大を逆畳み込み演算で実現する
* 全結合層と同等の処理を行う畳み込みに置き換えることができる
  * 全結合層では，出力がすべての入力と結びつく
  * 入力サイズが 32x10x10 (チャネル32，縦横それぞれ10)のデータに対する全結合層は32x10x10のフィルターサイズの畳み込み層に置き換えできる
  * 全結合層の出力ノード数が100の場合，畳み込み層では32x10x10のフィルターを100個用意すれば同じ処理を実現できる

### 8.4.3 画像キャプション生成
* 画像を与えると画像を説明する文章を自動生成する
* コンピュータビジョンと自然言語を融合した研究
  * マルチモーダル処理の一種
    * 複数の種類の情報を組み合わせて処理すること
* NIC(Neural Image Caption)
  * ディープラーニングによって画像キャプションを生成する代表的方法
  * ディープなCNNと自然言語を扱うためのRNNから構成される
    * RNN とは再帰的な繋がりをもつネットワーク
    * 自然言語や時系列データなどの連続性のあるデータによく用いられる
  * NICでは，画像からCNNによって特徴抽出し，RNNに特徴を渡す
  * RNNはCNNから得た特徴を初期値としてテキストを再帰的に生成する
* RNN(Recurrent Neural Network)
  * 再帰的なネットワーク構造をもつニューラルネットワーク
  * 再帰的構造により，以前に生成した情報から影響を受けることになる
    * これは過去の情報を記憶することになる
  * 自然言語や時系列データなど連続性のあるデータに対して過去の情報を記憶するように動作する

## 8.5 ディープラーニングの未来
* ディープラーニングの可能性と未来を感じさせる研究について

### 8.5.1 画像スタイル変換
* アーティストのような絵を描かせる研究
  * 例）「コンテンツ画像」と「スタイル画像」の2つの画像を入力して新しい画像を生成する研究
    * ネットワークの中間データがコンテンツ画像の中間データに近づくように学習することで，入力画像をコンテンツ画像の形状に似せることができる
    * スタイル行列という概念を導入し，スタイル行列のズレを小さくするように学習することで入力画像をスタイルに近づけることが実現される
    * （論文）[A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576)

### 8.5.2 画像生成
* 画像を用意することなく新しい画像を生成する研究
  * 学習する際には大量の画像を使用するが，描く際には画像を必要としない
* DCGAN(Deep Convolutional Generative Adversarial Network)
  * （論文）[Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434)
  * 画像が生成される過程をモデル化し，それを大量の画像で学習する
  * 2つのニューラルネットワークを利用している
    * Generator：本物そっくりの画像を生成する
    * Discriminator：画像が本物かどうかを判定する
  * 学習では両者を競わせるように行う(Generative Adversarial Network)
    * Generatorが本物のような画像を生成し，Discriminatorがその画像を判定する

#### 教師なし学習
* これまで見てきた問題は教師あり学習（supervised learning）と呼ばれる問題を扱ってきた
  * 画像データと教師データが対になって与えられたデータを利用する
* 教師データを与えない問題は教師なし学習（unsupervised learning）と呼ばれる
  * DCGANのように画像だけを大量に与える手法は教師なしに含まれる
* 従来はあまり活発に行われてこなかった
  * Deep Belief Network, Deep Boltzman Machine などが有名
* DCGANのような手法の登場により，さらなる発展が期待される

### 8.5.3 自動運転
* 周囲の環境を正しく認識する技術が重要な問題
* 日々刻々と変化する環境や縦横無尽に行き交う車や人を認識することがむずかしい問題
* 周囲の環境を認識する技術にディープラーニングが期待されている
* 例）[SegNet](http://mi.eng.cam.ac.uk/projects/segnet/)
  * CNNベースのネットワーク
  * 高精度に走路環境を認識する
  * 入力画像に対してセグメンテーションを行う

### 8.5.4 Deep Q-Network(強化学習: reinforcement learning)
* コンピュータが試行錯誤の過程から自律的に学習する
* 強化学習の枠組み：エージェントが環境の状況に応じて行動を選択し，行動によって環境が変化する
* 環境の変化によりエージェントは報酬を得る
* 強化学習の目的：より良い報酬が得られるようにエージェントの行動指針を決める
  * ここでいう報酬とは，見込みの報酬のこと
  * 見込みの報酬は最終的なゴールから逆算して決定される
* 代表的手法：Deep Q-Network (DQN)
  * Deep Mind 社による技術
  * Q学習と呼ばれる強化学習のアルゴリズムをベースとする
  * Q学習では最適な行動を決定するために「最適行動価値関数」と呼ばれる関数を決定する
  * 関数を決定するためにCNNを用いる
* 事例
  * テレビゲームの操作の学習
  * AlphaGoは内部でディープラーニングと強化学習が使われている

## 8.6 まとめ
* この章では，ディープなCNNを実装し，手書き数字認識で99％を超える認識結果を得た
* ネットワークをディープにすることのモチベーション
  * 多くの問題ではネットワークを深くすることで性能向上が期待できる
* 最近のディープラーニングはディープな方向へと向かっている
  * ILSVRCという画像認識コンペの最近の動向はディープラーニングによる手法が上位を独占し，使われるネットワークもディープになっている
  * 有名なネットワークとしてVGG, GoogLeNet, ResNetがある
* ディープラーニングのトレンド，実用例，高速化に向けた研究や未来の研究例の紹介
  * GPUや分散学習，ビット精度の削減などにより高速化を実現できる
  * ディープラーニングは物体認識以外にも物体検出やセグメンテーションに利用できる
  * ディープラーニングを用いた応用として画像のキャプション生成，画像生成，強化学習などがある．最近では自動運転への利用も期待されている
* 今後
  * ディープラーニングについてまだわかっていないことが多い
  * 新しい研究が次々と発表されている
  * これからも研究が活発に続けられ，新しい技術が現実化されてゆくと思われる

